# Projet Final Apache Spark

**Nom Etudiant : *Aminata*  

**Prenom Etudiant: *BA*  

**Classe : *Master 1 Big Data Analytics*  


## Description
Ce projet consiste à utiliser Apache Spark pour faire l'analyse et le traitement des données de **[San Francisco Fire Department Calls ](https://data.sfgov.org/Public-Safety/Fire-Department-Calls-for-Service/nuek-vuh3)** afin de fournir quelques KPI (*Key Performance Indicator*). Le **SF Fire Dataset** comprend les réponses aux appels de toutes les unités d'incendie. Chaque enregistrement comprend le numéro d'appel, le numéro d'incident, l'adresse, l'identifiant de l'unité, le type d'appel et la disposition. Tous les intervalles de temps pertinents sont également inclus. Étant donné que ce Dataset est basé sur les réponses et que la plupart des appels impliquent plusieurs unités, ainsi il existe plusieurs enregistrements pour chaque numéro d'appel. Les adresses sont associées à un numéro de bloc, à une intersection ou à une boîte d'appel, et non à une adresse spécifique.

**Plus de details sur la description des données cliquer sur ce [lien](https://data.sfgov.org/Public-Safety/Fire-Department-Calls-for-Service/nuek-vuh3)**

## Travail à faire.
L'objectif de ce projet est de comprendre le **SF Fire Dataset** afin de bien répondre aux questions en utilisant les codes Spark/Scala adéquates.

- Créer un repos git (public ou privé) et partager le repos avec mon mail (limahin10@gmail.com)
- Ecrire un code lisible et bien indenté 
- N'oublier pas de mettre en commentaire la justification de vos réponses sur les cellules Markdown. 


## Note:
- Le projet est personnel, c'est-à-dire chaque notebook ne concerne qu'un seul étudiant. 
- Deadline : **Jeudi 10 janvier 2021** (Aucune de dérogation ne sera acceptée)

### Chargement des données

Importation des packages Spark

In [1]:
import org.apache.spark.sql.types._ 
import org.apache.spark.sql.functions._ 
import spark.implicits._

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.155.159:4040
SparkContext available as 'sc' (version = 3.1.2, master = local[*], app id = local-1635942627948)
SparkSession available as 'spark'


import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import spark.implicits._


Nous allons jeter un coup d'oeil sur la structure des données avant de définir un schéma

In [2]:
!head -1 "datasets/sf-fire/sf-fire-calls.csv"

CallNumber,UnitID,IncidentNumber,CallType,CallDate,WatchDate,CallFinalDisposition,AvailableDtTm,Address,City,Zipcode,Battalion,StationArea,Box,OriginalPriority,Priority,FinalPriority,ALSUnit,CallTypeGroup,NumAlarms,UnitType,UnitSequenceInCallDispatch,FirePreventionDistrict,SupervisorDistrict,Neighborhood,Location,RowID,Delay



Vu que la taille de ces données est énormes, inferer le schema pour un très grande volumes de données s'avère un peu couteux. Nous allons ainsi définir un schema pour le Dataset.

In [3]:
val fireSchema = StructType(Array(StructField("CallNumber", IntegerType, true),
  StructField("UnitID", StringType, true),
  StructField("IncidentNumber", IntegerType, true),
  StructField("CallType", StringType, true),                  
  StructField("CallDate", StringType, true),      
  StructField("WatchDate", StringType, true),
  StructField("CallFinalDisposition", StringType, true),
  StructField("AvailableDtTm", StringType, true),
  StructField("Address", StringType, true),       
  StructField("City", StringType, true),       
  StructField("Zipcode", IntegerType, true),       
  StructField("Battalion", StringType, true),                 
  StructField("StationArea", StringType, true),       
  StructField("Box", StringType, true),       
  StructField("OriginalPriority", StringType, true),       
  StructField("Priority", StringType, true),       
  StructField("FinalPriority", IntegerType, true),       
  StructField("ALSUnit", BooleanType, true),       
  StructField("CallTypeGroup", StringType, true),
  StructField("NumAlarms", IntegerType, true),
  StructField("UnitType", StringType, true),
  StructField("UnitSequenceInCallDispatch", IntegerType, true),
  StructField("FirePreventionDistrict", StringType, true),
  StructField("SupervisorDistrict", StringType, true),
  StructField("Neighborhood", StringType, true),
  StructField("Location", StringType, true),
  StructField("RowID", StringType, true),
  StructField("Delay", FloatType, true)))

fireSchema: org.apache.spark.sql.types.StructType = StructType(StructField(CallNumber,IntegerType,true), StructField(UnitID,StringType,true), StructField(IncidentNumber,IntegerType,true), StructField(CallType,StringType,true), StructField(CallDate,StringType,true), StructField(WatchDate,StringType,true), StructField(CallFinalDisposition,StringType,true), StructField(AvailableDtTm,StringType,true), StructField(Address,StringType,true), StructField(City,StringType,true), StructField(Zipcode,IntegerType,true), StructField(Battalion,StringType,true), StructField(StationArea,StringType,true), StructField(Box,StringType,true), StructField(OriginalPriority,StringType,true), StructField(Priority,StringType,true), StructField(FinalPriority,IntegerType,true), StructField(ALSUnit,BooleanType,true)...


In [4]:
val sfFireFile = "datasets/sf-fire/sf-fire-calls.csv"
val fireDF = spark
  .read
  .schema(fireSchema)
  .option("header", "true")
  .csv(sfFireFile)

sfFireFile: String = datasets/sf-fire/sf-fire-calls.csv
fireDF: org.apache.spark.sql.DataFrame = [CallNumber: int, UnitID: string ... 26 more fields]


Nous allons mettre en cache le Dataframe

In [5]:
fireDF.cache()

res0: fireDF.type = [CallNumber: int, UnitID: string ... 26 more fields]


In [6]:
fireDF.count()

res1: Long = 175296


In [7]:
fireDF.printSchema()

root
 |-- CallNumber: integer (nullable = true)
 |-- UnitID: string (nullable = true)
 |-- IncidentNumber: integer (nullable = true)
 |-- CallType: string (nullable = true)
 |-- CallDate: string (nullable = true)
 |-- WatchDate: string (nullable = true)
 |-- CallFinalDisposition: string (nullable = true)
 |-- AvailableDtTm: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Zipcode: integer (nullable = true)
 |-- Battalion: string (nullable = true)
 |-- StationArea: string (nullable = true)
 |-- Box: string (nullable = true)
 |-- OriginalPriority: string (nullable = true)
 |-- Priority: string (nullable = true)
 |-- FinalPriority: integer (nullable = true)
 |-- ALSUnit: boolean (nullable = true)
 |-- CallTypeGroup: string (nullable = true)
 |-- NumAlarms: integer (nullable = true)
 |-- UnitType: string (nullable = true)
 |-- UnitSequenceInCallDispatch: integer (nullable = true)
 |-- FirePreventionDistrict: string (nullable = true)
 

In [8]:
fireDF.show(5)

+----------+------+--------------+----------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+-------------+-------+-------------+---------+--------+--------------------------+----------------------+------------------+--------------------+--------------------+-------------+---------+
|CallNumber|UnitID|IncidentNumber|        CallType|  CallDate| WatchDate|CallFinalDisposition|       AvailableDtTm|             Address|City|Zipcode|Battalion|StationArea| Box|OriginalPriority|Priority|FinalPriority|ALSUnit|CallTypeGroup|NumAlarms|UnitType|UnitSequenceInCallDispatch|FirePreventionDistrict|SupervisorDistrict|        Neighborhood|            Location|        RowID|    Delay|
+----------+------+--------------+----------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+------------

Filtrage des d'appels de type "Medical Incident"

In [9]:
val fewFireDF = fireDF
  .select("IncidentNumber", "AvailableDtTm", "CallType") 
  .where($"CallType" =!= "Medical Incident")

fewFireDF.show(5, false)

+--------------+----------------------+--------------+
|IncidentNumber|AvailableDtTm         |CallType      |
+--------------+----------------------+--------------+
|2003235       |01/11/2002 01:51:44 AM|Structure Fire|
|2003250       |01/11/2002 04:16:46 AM|Vehicle Fire  |
|2003259       |01/11/2002 06:01:58 AM|Alarms        |
|2003279       |01/11/2002 08:03:26 AM|Structure Fire|
|2003301       |01/11/2002 09:46:44 AM|Alarms        |
+--------------+----------------------+--------------+
only showing top 5 rows



fewFireDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [IncidentNumber: int, AvailableDtTm: string ... 1 more field]


### Question 1
**Combien de types d'appels distincts ont été passés ?**  
Pour être sûr, il ne faut pas compter les valeurs «nulles» dans la colonne.

In [47]:
// Reponse 1
/*
Ecrire ici votre code
*/
val noNullsDF = fireDF.na.drop(cols = Seq("CallType"))
val callFireDF = noNullsDF
    .select("CallType")
    .distinct()
    .count()

noNullsDF: org.apache.spark.sql.DataFrame = [CallNumber: int, UnitID: string ... 26 more fields]
callFireDF: Long = 30


### Question 2

**Quels types d'appels différents ont été passés au service d'incendie?**

In [11]:
// Reponse 2
/*
Ecrire ici votre code
*/
callFireDF.show(false)

+--------------------------------------------+
|CallType                                    |
+--------------------------------------------+
|Elevator / Escalator Rescue                 |
|Marine Fire                                 |
|Aircraft Emergency                          |
|Confined Space / Structure Collapse         |
|Administrative                              |
|Alarms                                      |
|Odor (Strange / Unknown)                    |
|Citizen Assist / Service Call               |
|HazMat                                      |
|Watercraft in Distress                      |
|Explosion                                   |
|Oil Spill                                   |
|Vehicle Fire                                |
|Suspicious Package                          |
|Extrication / Entrapped (Machinery, Vehicle)|
|Other                                       |
|Outside Fire                                |
|Traffic Collision                           |
|Assist Polic

### Question 3

**Trouver toutes les réponses ou les délais sont supérieurs à 5 minutes?**

*Indication
1. Renommer la colonne Delay -> ReponseDelayedinMins
2. Retourner un nouveau DataFrame
3. Afficher tous les appels où le temps de réponse à un site d'incendie a eu lieu après un retard de plus de 5 minutes

In [12]:
// Reponse 3
val newFireDF = fireDF.withColumnRenamed("Delay", "ResponseDelayedinMins")
/*
Completer le code
*/
val delayedFireDF = newFireDF
    .select("CallNumber", "AvailableDtTm", "CallType", "ResponseDelayedinMins")
    .where($"ResponseDelayedinMins" > 5)
    .show(false)

+----------+----------------------+-----------------------------+---------------------+
|CallNumber|AvailableDtTm         |CallType                     |ResponseDelayedinMins|
+----------+----------------------+-----------------------------+---------------------+
|20110315  |01/11/2002 04:34:23 PM|Medical Incident             |5.35                 |
|20120147  |01/12/2002 01:23:04 PM|Medical Incident             |6.25                 |
|20130013  |01/13/2002 01:51:15 AM|Medical Incident             |5.2                  |
|20140067  |01/14/2002 08:16:54 AM|Citizen Assist / Service Call|5.6                  |
|20140177  |01/14/2002 12:27:03 PM|Medical Incident             |7.25                 |
|20150056  |01/15/2002 06:53:38 AM|Medical Incident             |11.916667            |
|20150254  |01/15/2002 03:18:26 PM|Medical Incident             |5.116667             |
|20150265  |01/15/2002 03:55:41 PM|Medical Incident             |8.633333             |
|20150265  |01/15/2002 07:49:05 

newFireDF: org.apache.spark.sql.DataFrame = [CallNumber: int, UnitID: string ... 26 more fields]
delayedFireDF: Unit = ()


### Transformations des dates  
Maintenant nous allons d'abord:  
1. Transformer les dates de type String en Spark Timestamp afin que nous puissions effectuer des requêtes basées sur la date plus tard    
2. Retourner le Dataframe transformée  
3. Mettre en cache le nouveau DataFrame  

In [13]:
val fireTSDF = newFireDF
  .withColumn("IncidentDate", to_timestamp(col("CallDate"), "MM/dd/yyyy")).drop("CallDate") 
  .withColumn("OnWatchDate", to_timestamp(col("WatchDate"), "MM/dd/yyyy")).drop("WatchDate") 
  .withColumn("AvailableDtTS", to_timestamp(col("AvailableDtTm"), "MM/dd/yyyy hh:mm:ss a")).drop("AvailableDtTm")

fireTSDF.cache()

fireTSDF: org.apache.spark.sql.DataFrame = [CallNumber: int, UnitID: string ... 26 more fields]
res7: fireTSDF.type = [CallNumber: int, UnitID: string ... 26 more fields]


### Question 4
**Quels sont les types d'appels les plus courants?**

In [14]:
//Reponse 4
/*
Ecrire ici votre code
*/
val mostFrequentCallTypesDF = fireTSDF
    .groupBy($"CallType")
    .count()
    .orderBy($"count".desc)
    .show(false)

+-------------------------------+------+
|CallType                       |count |
+-------------------------------+------+
|Medical Incident               |113794|
|Structure Fire                 |23319 |
|Alarms                         |19406 |
|Traffic Collision              |7013  |
|Citizen Assist / Service Call  |2524  |
|Other                          |2166  |
|Outside Fire                   |2094  |
|Vehicle Fire                   |854   |
|Gas Leak (Natural and LP Gases)|764   |
|Water Rescue                   |755   |
|Odor (Strange / Unknown)       |490   |
|Electrical Hazard              |482   |
|Elevator / Escalator Rescue    |453   |
|Smoke Investigation (Outside)  |391   |
|Fuel Spill                     |193   |
|HazMat                         |124   |
|Industrial Accidents           |94    |
|Explosion                      |89    |
|Train / Rail Incident          |57    |
|Aircraft Emergency             |36    |
+-------------------------------+------+
only showing top

mostFrequentCallTypesDF: Unit = ()


### Question 5-a
**Quels sont boites postaux rencontrés dans les appels les plus courants?**

In [15]:
//Reponse 5-a
/*
Ecrire ici votre code
*/
val DF = fireTSDF
    .groupBy($"CallType", $"Zipcode")
    .count()
    .orderBy($"count".desc)
    .show(30, false)

+----------------+-------+-----+
|CallType        |Zipcode|count|
+----------------+-------+-----+
|Medical Incident|94102  |16130|
|Medical Incident|94103  |14775|
|Medical Incident|94110  |9995 |
|Medical Incident|94109  |9479 |
|Medical Incident|94124  |5885 |
|Medical Incident|94112  |5630 |
|Medical Incident|94115  |4785 |
|Medical Incident|94122  |4323 |
|Medical Incident|94107  |4284 |
|Medical Incident|94133  |3977 |
|Medical Incident|94117  |3522 |
|Medical Incident|94134  |3437 |
|Medical Incident|94114  |3225 |
|Medical Incident|94118  |3104 |
|Medical Incident|94121  |2953 |
|Medical Incident|94116  |2738 |
|Medical Incident|94132  |2594 |
|Structure Fire  |94110  |2267 |
|Medical Incident|94105  |2258 |
|Structure Fire  |94102  |2229 |
|Structure Fire  |94103  |2221 |
|Alarms          |94103  |2168 |
|Medical Incident|94108  |2162 |
|Structure Fire  |94109  |2160 |
|Alarms          |94102  |2140 |
|Medical Incident|94123  |1940 |
|Medical Incident|94131  |1853 |
|Alarms   

DF: Unit = ()


### Question 5-a
**Quels sont les quartiers de San Francisco dont les codes postaux sont 94102 et 94103?**

In [16]:
//Reponse 5-b
/*
Ecrire ici votre code
*/
val addressFireDF = fireTSDF
    .select("Neighborhood", "Zipcode")
    .where($"Zipcode" === 94102 or $"Zipcode" === 94103)
    .distinct()
    .show(false)

+------------------------------+-------+
|Neighborhood                  |Zipcode|
+------------------------------+-------+
|Potrero Hill                  |94103  |
|Western Addition              |94102  |
|Tenderloin                    |94102  |
|Nob Hill                      |94102  |
|Castro/Upper Market           |94103  |
|South of Market               |94102  |
|South of Market               |94103  |
|Hayes Valley                  |94103  |
|Financial District/South Beach|94102  |
|Mission Bay                   |94103  |
|Tenderloin                    |94103  |
|Financial District/South Beach|94103  |
|Hayes Valley                  |94102  |
|Mission                       |94103  |
+------------------------------+-------+



addressFireDF: Unit = ()


### Question 6
**Determiner le nombre total d'appels, ainsi que la moyenne, le minimum et le maximum du temps de réponse des appels?**

In [17]:
//Reponse 6
/*
Ecrire ici votre code
*/
fireTSDF.select("ResponseDelayedinMins").describe().show

+-------+---------------------+
|summary|ResponseDelayedinMins|
+-------+---------------------+
|  count|               175296|
|   mean|    3.892364154521585|
| stddev|    9.378286226254206|
|    min|          0.016666668|
|    max|              1844.55|
+-------+---------------------+



### Question 7-a
**Combien d'années distinctes trouve t-on dans ce Dataset?**  
Dans ce dataset nous avons des données comprises entre 2000-2018. Vous pouvez utilisez la fonction Spark `year()` pour les dates en Timestamp

In [23]:
//Reponse 7-a
/*
Ecrire ici votre code
*/
val test = fireTSDF.select(year($"IncidentDate")).distinct().sort(year($"IncidentDate")).show(false)

+------------------+
|year(IncidentDate)|
+------------------+
|2000              |
|2001              |
|2002              |
|2003              |
|2004              |
|2005              |
|2006              |
|2007              |
|2008              |
|2009              |
|2010              |
|2011              |
|2012              |
|2013              |
|2014              |
|2015              |
|2016              |
|2017              |
|2018              |
+------------------+



test: Unit = ()


### Question 7-b
**Quelle semaine de l'année 2018 a eu le plus d'appels d'incendie?**

In [69]:
//Reponse 7-b
/*
Ecrire ici votre code
*/
val mostCallsDF = fireTSDF
    .where(year($"IncidentDate") === 2018)
    .groupBy(month($"IncidentDate"))
    .count()
    .orderBy($"count".desc)
    .show(1)

+-------------------+-----+
|month(IncidentDate)|count|
+-------------------+-----+
|                 10| 1068|
+-------------------+-----+
only showing top 1 row



mostCallsDF: Unit = ()


### Question 8
**Quels sont les quartiers de San Francisco qui ont connu le pire temps de réponse en 2018?**

In [70]:
//Reponse 8
/*
Ecrire ici votre code
*/
import org.apache.spark.sql.functions.col
val test = fireTSDF
    .select($"Neighborhood", $"IncidentDate", $"ResponseDelayedinMins")
    .where(year($"IncidentDate") === 2018)
    .orderBy($"ResponseDelayedinMins".desc)
    .show(false)

+------------------------------+-------------------+---------------------+
|Neighborhood                  |IncidentDate       |ResponseDelayedinMins|
+------------------------------+-------------------+---------------------+
|Chinatown                     |2018-03-17 00:00:00|491.26666            |
|Financial District/South Beach|2018-01-20 00:00:00|406.63333            |
|Tenderloin                    |2018-03-22 00:00:00|340.48334            |
|Haight Ashbury                |2018-05-21 00:00:00|175.86667            |
|Bayview Hunters Point         |2018-04-17 00:00:00|155.8                |
|Financial District/South Beach|2018-06-08 00:00:00|135.51666            |
|Pacific Heights               |2018-05-03 00:00:00|129.01666            |
|Potrero Hill                  |2018-03-05 00:00:00|109.8                |
|Inner Sunset                  |2018-01-12 00:00:00|106.13333            |
|South of Market               |2018-04-21 00:00:00|94.71667             |
|Bayview Hunters Point   

import org.apache.spark.sql.functions.col
test: Unit = ()


### Question 9

**Comment stocker les données du Dataframe sous format de fichiers Parquet?**

In [71]:
//Reponse 9
/*
Ecrire ici votre code
*/
val outputPath = "datasets/sf-fire/sf-fire.parquet"

fireTSDF
    .write
    .mode("overwrite")
    .parquet(outputPath)

outputPath: String = datasets/sf-fire/sf-fire.parquet


### Question 10
**Comment relire les données stockée en format Parquet?**

In [72]:
//Reponse 10
/*
Ecrire ici votre code
*/
spark.read.parquet(outputPath).show

+----------+------+--------------+----------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+-------------+-------+-------------+---------+--------------+--------------------------+----------------------+------------------+--------------------+--------------------+-------------+---------------------+-------------------+-------------------+-------------------+
|CallNumber|UnitID|IncidentNumber|        CallType|CallFinalDisposition|             Address|City|Zipcode|Battalion|StationArea| Box|OriginalPriority|Priority|FinalPriority|ALSUnit|CallTypeGroup|NumAlarms|      UnitType|UnitSequenceInCallDispatch|FirePreventionDistrict|SupervisorDistrict|        Neighborhood|            Location|        RowID|ResponseDelayedinMins|       IncidentDate|        OnWatchDate|      AvailableDtTS|
+----------+------+--------------+----------------+--------------------+--------------------+----+-------+---------+-----------+----+-----------

## FIN